# ボートレーサーごとの統計量をdfに追加　その１
- 各枠からの1着率, 2着率, 3着率をそれぞれ計算し、特徴量として追加

## 1. 統計解析用のdfをロード
- `race_df`に格納
- 2_データ前処理.ipnbを参照

In [10]:
import pandas as pd
race_df = pd.read_pickle('../../data/arrangedData/data_formatted_for_statistic.pkl').reset_index(drop=True)
# 中身を確認
race_df.head()

,date,venue,raceNumber,枠,boatNo,motorNo,num_false_start,num_late_start,racer_class,racer_id,...,wave_height,weather,weight,wind_speed,タイム,着順,start_time,weather_曇り,weather_雨,weather_雪
0,2021-01-01,津,1.0,1,41,22,0,0,1,4926,...,2,晴,52.0,3,"1'16""1",2,0.24,0,0,0
1,2021-01-01,津,1.0,2,14,57,0,0,3,3644,...,2,晴,57.1,3,,6,0.39,0,0,0
2,2021-01-01,津,1.0,3,46,64,0,0,3,3881,...,2,晴,52.0,3,,5,0.35,0,0,0
3,2021-01-01,津,1.0,4,13,56,0,0,3,4675,...,2,晴,52.0,3,"1'15""1",1,0.16,0,0,0
4,2021-01-01,津,1.0,5,57,63,0,0,3,4358,...,2,晴,52.0,3,"1'17""2",4,0.22,0,0,0


## 2. これまでのレース結果から統計量を算出
- 各選手が、その枠からの1着率・２着率・3着率を算出し、それぞれ列に格納
- 選手ごと、枠ごと、日付ごとに、その前日までのレースを対象に統計量を算出

In [ ]:
from tqdm.notebook import tqdm

def calc_rank_i_ratio(data, rank_i):
    # rank_i着であった率を算出
    total_count = data.count()
    rank_i_count = (data == rank_i).sum()
    rank_i_ratio = rank_i_count / total_count
    
    return rank_i_ratio


date_list = sorted(race_df["date"].unique())
date_list = date_list[1:]

stocastic_df_list = []

for date in tqdm(date_list):

    the_date_df_list = []
    
    for i in range(1,4):
        race_df_before_the_date = race_df[race_df["date"] < date]

        # その日までのデータを集めたdfに対して、選手ごとにi着率を算出
        stocastic_df_ = race_df_before_the_date.groupby(["racer_id", "枠"])["着順"].apply(
            calc_rank_i_ratio, rank_i=i).reset_index()
        stocastic_df_.rename(columns={"着順": "{0}着率".format(i)}, inplace=True)
        
        # racer_idと枠をmultiindexとして指定
        stocastic_df_.set_index(["racer_id", "枠"], inplace=True)
        
        the_date_df_list.append(stocastic_df_)
    
    # その日までのレーサーごとの統計量を横方向に結合
    stocastic_df_date = pd.concat(the_date_df_list, axis=1)

    # dfにdate列を追加
    stocastic_df_date["date"] = date
    # その日についてまとめたデータをリストに格納
    stocastic_df_list.append(stocastic_df_date)

# あらゆる日付について縦方向にマージ
stocastic_df = pd.concat(stocastic_df_list)
# indexを振り直す
stocastic_df.reset_index(inplace=True)

stocastic_df

  0%|          | 0/84 [00:00<?, ?it/s]

In [13]:
# 統計値の範囲が十分かを考える
date = "2021/03/15"
for i in range(1,4):
        race_df_before_the_date = race_df[race_df["date"] < date]

        # その日までのデータを集めたdfに対して、選手ごとにi着率を算出
        stocastic_df_ = race_df_before_the_date.groupby(["racer_id", "枠"])["着順"].apply(
            calc_rank_i_ratio, rank_i=i).reset_index()

7
5
8
10
7
4
6
7
6
5
6
8
8
7
6
8
9
6
6
5
4
6
5
6
5
13
6
10
7
6
9
7
9
8
8
8
9
9
9
6
6
4
5
10
7
9
8
2
9
10
5
6
9
8
2
3
3
1
2
3
4
6
10
7
9
4
2
4
2
2
5
5
4
3
6
6
4
4
5
7
4
5
4
3
14
7
10
6
8
8
5
4
6
5
5
4
2
4
4
4
3
2
5
10
7
7
8
6
8
11
7
6
6
8
2
8
7
6
6
3
6
5
7
7
3
5
4
8
10
7
8
5
3
2
4
6
2
2
6
5
8
4
3
4
4
7
3
5
4
3
8
9
8
5
8
5
5
5
4
3
5
5
8
6
5
7
8
7
8
9
6
8
8
6
7
5
9
8
6
5
6
8
7
7
7
4
7
6
8
5
6
7
5
10
7
5
9
8
10
9
8
6
7
8
4
5
4
4
4
3
5
6
6
5
8
7
5
7
4
5
7
6
9
9
6
4
8
6
11
14
9
6
12
5
9
11
6
6
7
9
7
9
7
5
7
7
4
8
7
6
8
2
2
5
5
5
4
2
8
9
10
7
7
7
7
5
8
8
8
6
7
6
7
8
6
6
4
6
5
4
6
4
8
7
9
8
7
7
4
6
7
6
4
3
4
2
3
3
2
5
4
7
5
6
6
6
5
4
7
4
5
4
4
3
4
5
2
2
8
5
7
5
6
6
5
8
6
5
6
2
7
8
8
5
5
6
7
8
5
10
6
8
6
7
5
8
7
9
4
5
4
3
3
3
7
7
7
8
8
5
8
8
5
8
3
7
10
8
5
2
8
7
1
3
2
3
3
1
5
4
5
5
6
6
5
8
8
5
6
5
6
7
8
7
8
5
12
9
9
11
4
9
4
4
5
1
3
3
7
9
7
9
8
7
9
4
4
6
5
3
5
9
6
8
8
6
3
2
5
3
3
1
9
10
7
7
10
6
6
4
4
4
5
7
4
7
7
6
5
5
3
3
4
3
2
2
10
10
10
6
8
9
7
9
6
6
8
6
10
5
8
6
6
8
5
3
6
8
7
7
1
2
2
1
1
1


## 3. `race_df`に2で算出した統計量をマージ

In [9]:
merge_key_list = ["date", "枠", "racer_id"]
race_df_w_stocastic = pd.merge(race_df, stocastic_df,
                               left_on=merge_key_list, right_on=merge_key_list,
                               how="left"
                              )
race_df_w_stocastic

,date,venue,raceNumber,枠,boatNo,motorNo,num_false_start,num_late_start,racer_class,racer_id,...,weight,wind_speed,タイム,着順,weather_曇り,weather_雨,weather_雪,1着率,2着率,3着率
0,2021-01-01,津,1.0,1,41,22,0,0,1,4926,...,52.0,3,"1'16""1",2,0,0,0,NaN,NaN,NaN
1,2021-01-01,津,1.0,2,14,57,0,0,3,3644,...,57.1,3,,6,0,0,0,NaN,NaN,NaN
2,2021-01-01,津,1.0,3,46,64,0,0,3,3881,...,52.0,3,,5,0,0,0,NaN,NaN,NaN
3,2021-01-01,津,1.0,4,13,56,0,0,3,4675,...,52.0,3,"1'15""1",1,0,0,0,NaN,NaN,NaN
4,2021-01-01,津,1.0,5,57,63,0,0,3,4358,...,52.0,3,"1'17""2",4,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72307,2021-03-29,蒲 郡,11.0,2,77,67,0,0,3,3596,...,52.0,2,"1'53""1",4,0,0,0,0.000000,0.500000,0.100000
72308,2021-03-29,蒲 郡,11.0,3,40,19,0,0,1,4512,...,52.0,2,"1'49""1",1,0,0,0,0.000000,0.375000,0.125000
72309,2021-03-29,蒲 郡,11.0,4,48,39,0,0,1,4118,...,52.0,2,,6,0,0,0,0.090909,0.090909,0.090909
72310,2021-03-29,蒲 郡,11.0,5,66,38,1,0,2,4016,...,53.2,2,"1'52""4",3,0,0,0,0.000000,0.000000,0.250000


## 4. 3.で作った統計量入りのdfをpickleファイルにして保存

In [11]:
race_df_w_stocastic.to_pickle('../../data/arrangedData/race_df_w_stocastic_2.pkl')